In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount_retraite.csv', index_col=0)
df_test = pd.read_csv('datasets/SatisfactionClients/amazon_test1.csv', index_col=0)
df.sample(3)

,pays,note,titre,commentaire,Commentaire_filtre,titre_filtre
79248,FR,5,grand choix d'articles,4 févr. 2020Vérifiégrand choix d'articlesDate ...,"['févr', 'vérifiégrand', 'choix', 'articlesdat...","['grand', 'choix', 'articles']"
9801,FR,5,bien,18 mars 2022Sur invitationbienrien a redire me...,"['mars', 'invitationbienrien', 'redire', 'merc...",['bien']
9030,FR,5,très bien très vite livrer,2 avr. 2022Sur invitationtrès bien très vite l...,"['avr', 'invitationtrès', 'bien', 'très', 'vit...","['très', 'bien', 'très', 'vite', 'livrer']"


In [3]:
# séparation de la variable cible et des variables explicatives
X = df['Commentaire_filtre']
y = df['note']

In [4]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [5]:
# conversion des chaines de caratères en tokens numériques
# on ne prendra que les éléments répétés au moins 3 fois avec min_df=3
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=3)
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [7]:
# application du modèle de decisionTreeClassifier sur les tokens numériques et calcul des prédictions
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clf = DecisionTreeClassifier(max_depth=1)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test2 = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test2)

for i in cm_test2.index:
    for j in cm_test2.columns :
        if (i==j and cm_test2.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test2.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6412083333333334

              precision    recall  f1-score   support

           1       0.45      0.36      0.40      2057
           2       0.00      0.00      0.00       535
           3       0.00      0.00      0.00      1337
           4       0.00      0.00      0.00      4976
           5       0.66      0.97      0.78     15095

    accuracy                           0.64     24000
   macro avg       0.22      0.27      0.24     24000
weighted avg       0.45      0.64      0.53     24000



C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


predictions,1,5
données réelles,,
1,746,1311
2,104,431
3,144,1193
4,221,4755
5,452,14643


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,1,5
données réelles,,
1,9,11
2,10,10
3,8,12
4,4,16
5,3,17


predictions,1,5
données réelles,,
1,0.45,0.55
2,0.50,0.50
3,0.40,0.60
4,0.20,0.80
5,0.15,0.85


les prédictions correctes sont supérieures à 50% pour 5 étoile(s) avec 85.0 %


In [11]:
clf2 = DecisionTreeClassifier(max_depth=10)

clf2.fit(X_train, y_train)

y_pred2 = clf.predict(X_test)

print(clf2.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred2)
print(cr)

cm = pd.crosstab(y_test, y_pred2, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf2.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test2 = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test2)

for i in cm_test2.index:
    for j in cm_test2.columns :
        if (i==j and cm_test2.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test2.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6699166666666667

              precision    recall  f1-score   support

           1       0.45      0.36      0.40      2057
           2       0.00      0.00      0.00       535
           3       0.00      0.00      0.00      1337
           4       0.00      0.00      0.00      4976
           5       0.66      0.97      0.78     15095

    accuracy                           0.64     24000
   macro avg       0.22      0.27      0.24     24000
weighted avg       0.45      0.64      0.53     24000



C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


predictions,1,5
données réelles,,
1,746,1311
2,104,431
3,144,1193
4,221,4755
5,452,14643


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,1,2,4,5
données réelles,,,,
1,12,0,1,7
2,9,0,0,11
3,7,1,1,11
4,7,0,1,12
5,2,1,3,14


predictions,1,2,4,5
données réelles,,,,
1,0.60,0.00,0.05,0.35
2,0.45,0.00,0.00,0.55
3,0.35,0.05,0.05,0.55
4,0.35,0.00,0.05,0.60
5,0.10,0.05,0.15,0.70


les prédictions correctes sont supérieures à 50% pour 1 étoile(s) avec 60.0 %
les prédictions correctes sont supérieures à 50% pour 5 étoile(s) avec 70.0 %


In [12]:
from joblib import dump
dump(clf2, 'datasets/SatisfactionClients/model_dtc.jolib')

['datasets/SatisfactionClients/model_dtc.jolib']